In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import scipy.integrate as integrate
from __future__ import unicode_literals
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.mixture import GaussianMixture as gm
from multiprocessing import Pool
from sklearn.cluster import KMeans

In [2]:
def testujDecisionFunction(xTest, yTest, najDecFun, clf):
    wynik = clf.decision_function(xTest).reshape(yTest.shape)
    yTestWiekszeOdZera = yTest > 0
    if (najDecFun[1]):
        return np.sum((wynik > najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]
    return np.sum((wynik < najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]

def najlepszeDecFun(yTrain, wynikDecFun):
    yTrainWiekszeOdZera = yTrain > 0
    tabWynikow = []
    for i in range(wynikDecFun.shape[0]):
        wy = np.sum((wynikDecFun < wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
        wy = np.sum((wynikDecFun > wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
    pozycjaMaxArg = np.argmax(tabWynikow)
    if (pozycjaMaxArg % 2 == 0):
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 0)
    else:
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 1)

def crossValidationPoKolei2(clf, data, dataClass, k, p):
    size = data.shape[0]
    arr = np.arange(size)
    err = 0
    n = int(size / (k+1))
    trainCount = int(2 * n * p)
    for i in range(k):
        trainMask = np.arange(i*n, i*n + trainCount)
        testMask = np.arange(i*n + trainCount, (i+2)*n)
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        err += wynik
    return float(err) / k

def standaryzowanie(tab):
    tabS = prep.StandardScaler().fit_transform(tab)
    tabN = prep.Normalizer().fit_transform(tab)
    tabSN = prep.Normalizer().fit_transform(tabS)
    tabNS = prep.StandardScaler().fit_transform(tabN)
    return tab, tabS, tabN, tabSN, tabNS

def crossValidationPoKolei(clf, data, dataClass):
    err = 0
    k = len(data)
    for i in range(k):
        X_train = data[i][0]
        y_train = dataClass[i][0]
        X_test = data[i][1]
        y_test = dataClass[i][1]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        err += wynik
    return float(err) / k

def generujPodzialy(tab, p = 0.75):
    podzial = []
    n = len(tab)
    k = int(10*n/100)
    dlugosc = int(4/10*n)
    dlugoscUczaca = int(4/10*p*n)
    for i in range(6):
        uczace = tab[np.arange(i*k, min(i*k + dlugoscUczaca, n))]
        testowe = tab[np.arange(min(i*k + dlugoscUczaca, n), min((i*k + dlugosc), n))]
        podzial.append([uczace, testowe])
    return podzial

In [3]:
gammas = [1e-6, 1e-4, 1e-2, 1, 1e2]
Cs = [1e-6, 1e-4, 1e-2, 1, 1e2, 1e4]
tabParametrow = np.zeros((len(gammas)*len(Cs),2), dtype=np.float64)
i = 0
for gamma in gammas:
    for C in Cs:
        tabParametrow[i][0] = gamma
        tabParametrow[i][1] = C
        i += 1

In [4]:
zmiana = np.load("zmiana.npy")
podzialZmiana = generujPodzialy(zmiana)

In [5]:
def test(tablica, tablica2, tablica3, tablica4, tablica5, podzialZmiana, svm):
    wynik1 = crossValidationPoKolei(svm, tablica, podzialZmiana)
    wynik2 = crossValidationPoKolei(svm, tablica2, podzialZmiana)
    wynik3 = crossValidationPoKolei(svm, tablica3, podzialZmiana)
    wynik4= crossValidationPoKolei(svm, tablica4, podzialZmiana) 
    wynik5 = crossValidationPoKolei(svm, tablica5, podzialZmiana) 
    return wynik1, wynik2, wynik3, wynik4, wynik5

def wylicz(TAB, podzialZmiana, tabParametrow):
    tab, tabS, tabN, tabSN, tabNS = standaryzowanie(TAB)
    podzialTab = generujPodzialy(tab)
    podzialTabS = generujPodzialy(tabS)
    podzialTabN = generujPodzialy(tabN)
    podzialTabSN = generujPodzialy(tabSN)
    podzialTabnS = generujPodzialy(tabNS)
    
    tabWynikow = np.zeros((len(tabParametrow),5), dtype=np.float32)
    i = 0
    for parametry in tabParametrow:
        gamma = parametry[0]
        C = parametry[1]
        svm = SVC(class_weight='balanced', gamma = gamma, kernel = 'rbf', C = C, random_state=1)
        tabWynikow[i][0], tabWynikow[i][1], tabWynikow[i][2], tabWynikow[i][3], tabWynikow[i][4] = test(
            podzialTab, podzialTabS, podzialTabN, podzialTabSN, podzialTabnS, podzialZmiana, svm)
        i  += 1
    return tabWynikow

def wypisz(tabWynikow):
    print('Czyste     S      N     S->N    N->S')
    i = 0
    for parametry in tabParametrow:
        gamma = parametry[0]
        C = parametry[1]
        wynik1, wynik2, wynik3, wynik4, wynik5 = tabWynikow[i]
        odpowiedz = "{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5)
        if (wynik1 > 0.52 or wynik2 > 0.52 or wynik3 > 0.52 or wynik4 > 0.52 or wynik5 > 0.52):
            odpowiedz = "\x1b[31m\"{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}\"\x1b[0m".format(wynik1, wynik2, wynik3, wynik4, wynik5)
        print(odpowiedz, "- dla: g:", gamma, "C:", C) 
        i += 1

In [ ]:
t = time.time()
gmm_5_M_fPierw = np.load("gmm_5_M_fPierw.npy")
wynikiGmm_5_M_fPierw = wylicz(gmm_5_M_fPierw, podzialZmiana, tabParametrow)
print (time.time() -t)
np.save("wynikiGmm_5_M_fPierwTest2", wynikiGmm_5_M_fPierw)
wypisz(wynikiImbalance)